In [1]:
import os

In [2]:
%pwd

'c:\\ArmaanSeth\\python\\MLOPS\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\ArmaanSeth\\python\\MLOPS'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ArmaanSeth/MLOPS.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ArmaanSeth"
os.environ["MLFLOW_TRACKING_PASSWORD"]="38d8f0641386b6e2339ac0aae6ce344cfa756fa5"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [27]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_configuration(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet 
        schema = self.schema.TARGET
        create_directories([config.root_dir])
        print(config.keys())
        model_evaluation_config=ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = params,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/ArmaanSeth/MLOPS.mlflow"
        )
        return model_evaluation_config

In [28]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib

In [32]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_to_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        testx = test_data.drop([self.config.target_column],axis=1)
        testy = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_quantities = model.predict(testx)
            (rmse, mae, r2) = self.eval_metrics(testy, predicted_quantities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            # model registry does not work with file store
            if tracking_url_type_store != 'file':

                # register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            
            else:
                mlflow.sklearn.log_model(model, "model")

In [34]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_configuration()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_to_mlflow()
except Exception as e:
    raise e

[2023-12-25 17:00:16,518: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-25 17:00:16,519: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-25 17:00:16,524: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-25 17:00:16,525: INFO: common: created directory at: artifacts]
[2023-12-25 17:00:16,527: INFO: common: created directory at: artifacts/model_evaluation]
dict_keys(['root_dir', 'test_data_path', 'model_path', 'metric_file_name'])
[2023-12-25 17:00:17,137: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/12/25 17:00:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.
